In [1]:
import ratterdam_CoreDataStructures as Core
import ratterdam_ParseBehavior as Parse
import numpy as np
from scipy.stats import sem
import utility_fx as util
import os
import matplotlib.gridspec as gridspec
from matplotlib import pyplot as plt
import ratterdam_Defaults as Def
import ratterdam_visBasic as Vis
import RateMapClass_William_20190308 as RateMapClass
import williamDefaults as wmDef

In [2]:
%qtconsole

In [43]:
class Unit_Basic():
    """
    This class is a wrapper to hold position and spikes
    """
    
    def __init__(self, clustName, datafile, building):
        self.clustName = clustName
        self.datafile = datafile
        self.building = building
        self.load_data()
        
    def load_data(self):
        pos = util.read_pos(self.datafile)
        hdr, rec = util.readNEV(self.datafile)
        ts = np.asarray(sorted(list(pos.keys())))
        #start, stop = 1614284278,4576957801
        #ts = ts[(ts>start)&(ts<stop)]

        # ratterdam is orientated differently in
        # krieger vs macaulay so the adjustment to flip 
        # the image to cartesian coords (i.e. 0,0 is lower left)
        # is different
        if self.building == 'macaulay':
            posx = [pos[i][0] for i in ts]
            posy = [480-pos[i][1] for i in ts]
        elif self.building == 'krieger':
            posx = [640 - pos[i][0] for i in ts]
            posy = [pos[i][1] for i in ts]

        position = np.column_stack((ts,posx,posy))
        position = position[position[:,2]>75]
        clust = util.read_clust(self.datafile+self.clustName)
        clust = np.asarray(clust)
        
        spikexy = util.getPosFromTs(clust,position)
        spikes = np.column_stack((clust,spikexy))
        
        self.position = position
        self.spikes = spikes
        
        if self.building == 'krieger':
            self.alleyBounds = Def.alleyBounds_fromPlot2
        elif self.building == 'macaulay':
            self.alleyBounds = Def.alleyBounds
        

In [46]:
datafile = 'E:\\Ratterdam\\R781\\Ratterdam_D4_190411\\'
rat = "R781"
building = "macaulay"
clustName = "TT3\\cl-maze1.2"
unit = Unit_Basic(clustName, datafile, building)
rm = RateMapClass.RateMap(unit)

C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\RateMapClass_William_20190308.py:207: RuntimeWarning: invalid value encountered in greater_equal
  aboveThreshold = np.where(rateMap >= fieldThreshold, True, False)


In [18]:
def getVisits(data, maxgap=2*1e6):
    """
    """
    data.sort()
    groups = [[data[0]]]
    for x in data[1:]:
        if abs(x - groups[-1][-1]) <= maxgap:
            groups[-1].append(x)
        else:
            groups.append([x])
    return groups

In [19]:
cnames = ['black', 'blue', 'green', 'red', 'firebrick', 'cornflowerblue', 'orchid', 'darkcyan', 'midnightblue', 'saddlebrown', 'darkviolet', 'seagreen', 'indianred', 'goldenrod', 'orange', 'olive']
binWidth = wmDef.binWidth
from matplotlib import path
cmap = util.makeCustomColormap()

In [52]:
fig, ax = plt.subplots(2,1,figsize=(10,10))
colors = np.random.choice(cnames, len(rm.PF), replace=False)

fig.axes[0].imshow(rm.rateMap2D, origin='lower', aspect='auto', interpolation='None', cmap=cmap, vmax=np.nanpercentile(rm.rateMap2D, 98),
            extent=[wmDef.xedges[0], wmDef.xedges[-1], wmDef.yedges[0], wmDef.yedges[-1]])
fig.axes[0].set_title(f"Rat {rat} Unit {clustName}, color map cutoff = {round(np.nanpercentile(rm.rateMap2D, 98),2)}Hz", fontsize=24)


for i,pf in enumerate(rm.PF[:6]):
    contour = path.Path(list(zip(pf.perimeter[1]*binWidth+binWidth/2, pf.perimeter[0]*binWidth+binWidth/2)))
    PinC = unit.position[contour.contains_points(unit.position[:,1:])]
    posVisits = getVisits(PinC[:,0])
    field_FR = []
    field_TS = [] # take middle ts 
    for visit in posVisits:
        spk = unit.spikes[np.logical_and(unit.spikes[:,0] > visit[0], unit.spikes[:,0] < visit[-1])]
        field_FR.append((spk.shape[0]/len(visit))*30)
        field_TS.append(((visit[0]-unit.position[0,0])/1e6)/60)
    field_FR = util.weird_smooth(np.asarray(field_FR), 2)
    
    if i==1 or i==2:
        fig.axes[1].plot(field_TS, field_FR, color=colors[i], marker='.',linewidth=2)
    else:
        fig.axes[1].plot(field_TS, field_FR, color=colors[i], marker='.',alpha=0.4)
    fig.axes[1].text(field_TS[0],field_FR[0],f"{i}",color=colors[i])
    fig.axes[0].plot(pf.perimeter[1]*binWidth+binWidth/2, pf.perimeter[0]*binWidth+binWidth/2, marker='s', color=colors[i], markersize=3, linestyle='') #binWidth+5 is to align the markers because the extent is set previously to the xedges/yedges with imshow      

fig.axes[0].axis('equal')
fig.axes[0].set_ylim([0,480])
fig.axes[0].set_xlim([0, 640])
fig.axes[0].set_xticks([])
fig.axes[0].set_yticks([])
fig.axes[0].spines['top'].set_visible(False)
fig.axes[0].spines['right'].set_visible(False)
fig.axes[0].spines['bottom'].set_visible(False)
fig.axes[0].spines['left'].set_visible(False)

fig.axes[1].tick_params(axis='y', labelsize=20)
fig.axes[1].tick_params(axis='x', labelsize=20)
fig.axes[1].set_xlabel("Time in session (mins)", fontsize=24)
fig.axes[1].set_ylabel("Firing Rate (Hz, smoothed)", fontsize=24)
fig.axes[1].spines['right'].set_visible(False)
fig.axes[1].spines['top'].set_visible(False)
    
    

In [10]:
%matplotlib qt5

In [73]:
extent = 0.25*1e6
aheadboundaries = []
behindboundaries = []
spikePos = np.empty((0,2))

for spike in unit.spikes:
    spikeTs = spike[0]
    spikePos = np.vstack((spikePos, spike[1:]))
    posIdx = np.where(unit.position[:,0] == util.takeClosest(unit.position[:,0], spikeTs))[0][0]

    #ahead
    tsInt = util.getTsinInverval(unit.position[:,0], unit.position[posIdx][0], unit.position[posIdx][0]+extent)
    aheadpos = util.getPosFromTs(tsInt,unit.position)
    aheadpos  = np.asarray([i for i in aheadpos if i[1]>5])
    
    tsInt = util.getTsinInverval(unit.position[:,0], unit.position[posIdx][0]-extent, unit.position[posIdx][0])
    behindpos = util.getPosFromTs(tsInt,unit.position)
    behindpos  = np.asarray([i for i in behindpos if i[1]>5])

    
    aheadboundaries.append(aheadpos)
    behindboundaries.append(behindpos)



In [74]:
fig,ax = plt.subplots(1,2, figsize=(10,10))
for a in aheadboundaries:
    if a.shape[0] > 0:
        ax[0].plot(a[:,0], a[:,1],color='k',alpha=0.25)
    #ax[0].scatter(a[:,0], a[:,1],c='k',alpha=0.25, s=2)
ax[0].set_title(f"Ahead {extent/1e6}s")
for a in behindboundaries:
    if a.shape[0] > 0:
        ax[1].plot(a[:,0], a[:,1],color='k',alpha=0.25)
    #ax[1].scatter(a[:,0], a[:,1],c='k',alpha=0.25, s=2)
ax[1].set_title(f"Behind {extent/1e6}s")
plt.suptitle(f"Unit {clustName}")


Text(0.5,0.98,'Unit TT12\\cl-maze1.5')

In [232]:
b = 10
stack = np.zeros((b,b,0))
for a in behindboundaries:
    if a.shape[0] > 0:
        h = np.histogram2d(a[:,0], a[:,1], bins=[b,b])[0]
        stack = np.dstack((stack,h.T))
sm = np.nanmean(stack,axis=2)

In [56]:
for i in range(len(aheadboundaries)): # at this point ahead and behind have same len
    if aheadboundaries[i].shape[0] > 0:
        aheadboundaries[i] = aheadboundaries[i] - spikePos[i]
    if behindboundaries[i].shape[0] > 0:
        behindboundaries[i] = behindboundaries[i] - spikePos[i]
        
aheadboundaries = [i for i in aheadboundaries if i.shape[0]>0]
behindboundaries = [i for i in behindboundaries if i.shape[0]>0]
    
for i in range(len(aheadboundaries)):
    aheadboundaries[i] = aheadboundaries[i][ (aheadboundaries[i][:,0]> -130) & (aheadboundaries[i][:,0]<130) & 
                                            (aheadboundaries[i][:,1]>-130) & (aheadboundaries[i][:,1]<130)  ]
    
for i in range(len(behindboundaries)):
    behindboundaries[i] = behindboundaries[i][ (behindboundaries[i][:,0]> -130) & (behindboundaries[i][:,0]<130) & 
                                            (behindboundaries[i][:,1]>-130) & (behindboundaries[i][:,1]<130)  ]
    
ahead = np.concatenate(aheadboundaries, axis=0)
behind = np.concatenate(behindboundaries,axis=0)

aheadhist = np.histogram2d(ahead[:,1], ahead[:,0],bins=[50,50])[0]
behindhist = np.histogram2d(behind[:,1], behind[:,0], bins=[50,50])[0]

In [57]:
fig, ax = plt.subplots(2,2)

for a in aheadboundaries:
    fig.axes[0].plot(a[:,0], a[:,1],color='k',alpha=0.1)
    
for b in behindboundaries:
    fig.axes[1].plot(b[:,0], b[:,1],color='k',alpha=0.1)
    
fig.axes[2].imshow(aheadhist, origin='lower', aspect='auto', interpolation='None', vmax=300, cmap=cmap)
fig.axes[3].imshow(behindhist, origin='lower', aspect='auto', interpolation='None', vmax=300, cmap=cmap)

fig.axes[0].set_title(f"Trajectory Ahead, {extent/1e6}s", fontsize=16)
fig.axes[1].set_title(f"Trajectory Behind, {extent/1e6}s", fontsize=16)
fig.axes[0].set_ylabel("Overlay", fontsize=16)
fig.axes[2].set_ylabel("Histogram", fontsize=16)

Text(0,0.5,'Histogram')

### Blocking Experiments - Preliminary Analysis of Rate maps by session, field dynamics, etc

In [1]:
## For days when mapping session was separate NL session, need to load as two 'cells' and join separately


In [3]:
datafile1 = "E:\\Ratterdam\\R781\\alley blocking\\2019-05-01_15-13-18\\"
datafile2 = "E:\\Ratterdam\\R781\\alley blocking\\2019-05-01_16-47-15\\"
unitname = "TT5\\cl-maze1.6"

position1 = util.load_position(datafile1)
position2 = util.load_position(datafile2)


clust = util.read_clust(datafile1+unitname)
clust = np.asarray(clust)
spikexy = util.getPosFromTs(clust,position1)
spikes1 = np.column_stack((clust, spikexy))

clust = util.read_clust(datafile2+unitname)
clust = np.asarray(clust)
spikexy = util.getPosFromTs(clust,position2)
spikes2 = np.column_stack((clust, spikexy))


In [4]:
_, events1 = util.readNEV(datafile1)
_, events2 = util.readNEV(datafile2)
print(events1)
print("\n")
print(events2)

[ (0, 0, 2, 3625998757, 19, 0, 0, 0, 0, [0, 0, 0, 0, 0, 0, 0, 0], b'Starting Recording')
 (0, 0, 2, 3627274707,  4, 0, 0, 0, 0, [0, 0, 0, 0, 0, 0, 0, 0], b'begin sleep 1 ')
 (0, 0, 2, 5473175466,  4, 0, 0, 0, 0, [0, 0, 0, 0, 0, 0, 0, 0], b'end sleep 1 ')
 (0, 0, 2, 5475128984, 19, 0, 0, 0, 0, [0, 0, 0, 0, 0, 0, 0, 0], b'Stopping Recording')
 (0, 0, 2, 5762078900, 19, 0, 0, 0, 0, [0, 0, 0, 0, 0, 0, 0, 0], b'Starting Recording')
 (0, 0, 2, 5765316831,  4, 0, 0, 0, 0, [0, 0, 0, 0, 0, 0, 0, 0], b'begin session mapping')
 (0, 0, 2, 6682079830,  4, 0, 0, 0, 0, [0, 0, 0, 0, 0, 0, 0, 0], b'end session mapping')
 (0, 0, 2, 6682986499, 19, 0, 0, 0, 0, [0, 0, 0, 0, 0, 0, 0, 0], b'Stopping Recording')]


[ (0, 0, 2,  9257860513, 19, 0, 0, 0, 0, [0, 0, 0, 0, 0, 0, 0, 0], b'Starting Recording')
 (0, 0, 2,  9258915030,  4, 0, 0, 0, 0, [0, 0, 0, 0, 0, 0, 0, 0], b'begin anti-blocking session')
 (0, 0, 2, 10178615896,  4, 0, 0, 0, 0, [0, 0, 0, 0, 0, 0, 0, 0], b'end anti-blocking session')
 (0, 0, 2, 101

In [5]:
events = {"mapping":[5765316831, 6682079830], "antiblock":[9258915030, 10178615896], "block":[10243492271, 11222986475], "post":[11290616605, 12509462392]}

In [6]:
class UnitRep():
    
    def __init__(self):
        self.position = None
        self.spikes = None

In [7]:
unitdata = {}

unitdata["mapping"] = UnitRep()
p = position1[np.where(np.logical_and(position1[:,0] > events["mapping"][0], position1[:,0] < events["mapping"][1]))]
s = spikes1[np.where(np.logical_and(spikes1[:,0] > events["mapping"][0], spikes1[:,0] < events["mapping"][1]))]
unitdata["mapping"].position = p
unitdata["mapping"].spikes = s
unitdata["mapping"].RM = util.makeRM(s,p, bins=[30,50])
unitdata["mapping"].rep = RateMapClass.RateMap(unitdata["mapping"])

for sess in ["antiblock", "block", "post"]:
    unitdata[sess] = UnitRep()
    p = position2[np.where(np.logical_and(position2[:,0] > events[sess][0], position2[:,0] < events[sess][1]))]
    s = spikes2[np.where(np.logical_and(spikes2[:,0] > events[sess][0], spikes2[:,0] < events[sess][1]))]
    unitdata[sess].position = p
    unitdata[sess].spikes = s
    unitdata[sess].RM = util.makeRM(s,p, bins=[30,50])
    unitdata[sess].rep = RateMapClass.RateMap(unitdata[sess])


C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\utility_fx.py:493: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\utility_fx.py:493: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\RateMapClass_William_20190308.py:207: RuntimeWarning: invalid value encountered in greater_equal
  aboveThreshold = np.where(rateMap >= fieldThreshold, True, False)


In [11]:
fig, ax = plt.subplots(2,2)
m = np.vstack((unitdata["mapping"].RM, unitdata["antiblock"].RM, unitdata["block"].RM, unitdata["post"].RM))
mymax = np.nanpercentile(m, 97)
for sess,axis in zip(["mapping", "antiblock", "block", "post"], fig.axes):
    axis.imshow(unitdata[sess].RM, origin='lower', aspect='auto', interpolation='none', cmap=cmap,vmax=mymax)
    axis.set_title(sess)
fig.suptitle(f"{unitname}, FR saturation at {round(mymax,2)}Hz")

Text(0.5,0.98,'TT5\\cl-maze1.6, FR saturation at 28.63Hz')

In [145]:
fig, ax = plt.subplots(2,4,figsize=(10,10))
pct = 98

for sess,a in zip(["mapping", "antiblock", "block", "post"],[0,1,2,3]):
    
    cnames = np.random.choice(list(plt.cm.colors.CSS4_COLORS.keys()),len(unitdata[sess].rep.PF))

    unitdata[sess].rep = RateMapClass.RateMap(unitdata[sess])
    ax[0,a].imshow(unitdata[sess].rep.rateMap2D, origin='lower', aspect='auto', interpolation='None', cmap=cmap, vmax=np.nanpercentile(unitdata[sess].rep.rateMap2D, pct),
            extent=[wmDef.xedges[0], wmDef.xedges[-1], wmDef.yedges[0], wmDef.yedges[-1]])
    colors = np.random.choice(cnames, len(unitdata[sess].rep.PF), replace=False)
    
    for i,pf in enumerate(unitdata[sess].rep.PF):
        #pf = unitdata[sess].rep.PF[4]
        contour = path.Path(list(zip(pf.perimeter[1]*wmDef.binWidth+wmDef.binWidth/2, pf.perimeter[0]*wmDef.binWidth+wmDef.binWidth/2)))
        PinC = unitdata[sess].position[contour.contains_points(unitdata[sess].position[:,1:])]
        posVisits = util.getVisits(PinC[:,0])
        field_FR = []
        field_TS = [] # take middle ts 
        for visit in posVisits:
            spk = unitdata[sess].spikes[np.logical_and(unitdata[sess].spikes[:,0] > visit[0], unitdata[sess].spikes[:,0] < visit[-1])]
            field_FR.append((spk.shape[0]/len(visit))*30)
            field_TS.append(visit[0])
        field_FR = util.weird_smooth(np.asarray(field_FR), 2)
        ax[1,a].plot(field_TS, field_FR, color=colors[i], marker='.')
        ax[0,a].plot(pf.perimeter[1]*wmDef.binWidth+wmDef.binWidth/2, pf.perimeter[0]*wmDef.binWidth+wmDef.binWidth/2, marker='s', color=colors[i], markersize=3, linestyle='') #binWidth+5 is to align the markers because the extent is set previously to the xedges/yedges with imshow      


NameError: name 'unitdata' is not defined

In [156]:
# extent = 1e6
# session = unitdata['mapping']
# pf = session.rep.PF[4]
# contour = path.Path(list(zip(pf.perimeter[1]*wmDef.binWidth+wmDef.binWidth/2, pf.perimeter[0]*wmDef.binWidth+wmDef.binWidth/2)))
# PinC = session.position[contour.contains_points(session.position[:,1:])]
# posVisits = util.getVisits(PinC[:,0])
# field_FR = []
# visits = []
# spikes = []
# for visit in posVisits:
#     visit = [visit[0]-extent] + visit + [visit[-1]+extent]
#     spk = session.spikes[np.logical_and(session.spikes[:,0] > visit[0], session.spikes[:,0] < visit[-1])]
#     field_FR.append((spk.shape[0]/len(visit))*30)
#     v = util.getPosFromTs(visit,session.position)
#     visits.append(v)
#     spikes.append(spk)

In [12]:
ratplot = Vis.BasicRateMaps()

In [27]:
sesstype = 'mapping'
alley = 6
session = unitdata[sesstype]
alleyTracking = {i:np.empty((0,3)) for i in range(17)}
for entry in session.position:
    for a in Def.alleyBounds.keys():
        if entry[1] > Def.alleyBounds[a][0][0] and entry[1] < Def.alleyBounds[a][0][1] and entry[2] > Def.alleyBounds[a][1][0] and entry[2] < Def.alleyBounds[a][1][1]:
            alleyTracking[a] = np.vstack((alleyTracking[a], entry))

posVisits = util.getVisits(alleyTracking[alley-1][:,0])
visits = []
spikes = []
extent=3e6
for visit in posVisits:
    begin,end = visit[0]-extent, visit[-1]+extent
    spk = session.spikes[np.logical_and(session.spikes[:,0] >= begin, session.spikes[:,0] <= end)]
    v = session.position[np.logical_and(session.position[:,0] >= begin, session.position[:,0] <= end)]
    visits.append(v)
    spikes.append(spk)

In [21]:
fig, ax = plt.subplots(int(np.ceil(len(visits)/5)),5,figsize=(10,10))
for i in range(len(visits)):
    v = visits[i]
    s = spikes[i]
    for x in range(1,18):
        ratplot.drawAlleyBounds(fig.axes[i],x)
    fig.axes[i].plot(v[:,1], v[:,2],color='k',alpha=0.5)
    fig.axes[i].scatter(s[:,1], s[:,2],c='r',alpha=0.7,s=2)
    fig.axes[i].scatter(v[0,1], v[0,2],c='g',marker='x')
plt.suptitle(f"Alley {alley} Session {sesstype}, extent {extent/1e6}s")
    #fig.axes[i].plot(pf.perimeter[1]*wmDef.binWidth+wmDef.binWidth/2, pf.perimeter[0]*wmDef.binWidth+wmDef.binWidth/2, marker='s', color='k', markersize=1,linestyle='')

Text(0.5,0.98,'Alley 8 Session mapping, extent 3.0s')

In [15]:
# santiy check on spike position wrt traj
session = unitdata['post']
pf = session.rep.PF[4]
contour = path.Path(list(zip(pf.perimeter[1]*wmDef.binWidth+wmDef.binWidth/2, pf.perimeter[0]*wmDef.binWidth+wmDef.binWidth/2)))
PinC = session.position[contour.contains_points(session.position[:,1:])]
posVisits = util.getVisits(PinC[:,0])
visits = []
spikes = []
for visit in posVisits:
    spk = session.spikes[np.logical_and(session.spikes[:,0] >= visit[0], session.spikes[:,0] <= visit[-1])]
    v = util.getPosFromTs(visit,session.position)
    visits.append(v)
    spikes.append(spk)

In [28]:
# ratemaps
rms = []
_max = 0
for i in range(len(visits)):
    rm = util.makeRM(spikes[i], visits[i],bins=[15,10])
    rms.append(rm)
    if np.nanpercentile(rm,97) > _max:
        _max = np.nanpercentile(rm,97)
        
fig, ax = plt.subplots(int(np.ceil(len(visits)/5)),5,figsize=(10,10))
for i,rm in enumerate(rms):
    fig.axes[i].imshow(rm, aspect='auto',interpolation='None',cmap=cmap,vmax=20,origin='lower')
plt.title(sesstype)

C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\utility_fx.py:493: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\utility_fx.py:493: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*30


Text(0.5,1,'mapping')

## Looking at Rewards w.r.t. repeating field dynamics

In [328]:
hdr, rec = util.readNEV(datafile)
rewards = [i[3] for i in rec if b'0x0080' in i[10]] # check that the ttl code you're using is right. there is a util fx (or parse i forget) that does it as longas it doesnt actually change in a session
alley = 14
rpos = util.getPosFromTs(rewards,unit.position)
ab = Def.alleyBounds[alley-1]
idx=np.where((rpos[:,0]>ab[0][0]) & (rpos[:,0]<ab[0][1]) & (rpos[:,1]>ab[1][0]) & (rpos[:,1]<ab[1][1]))
for i in idx[0]:
    plt.vlines(rewards[i],0,20,color='g')
    

In [269]:
def calcAlleyCenter(ab):
    """
    pass in alleybounds defined as [[x1,x2],[y1,y2]]
    """
    x1,x2,y1,y2 = ab[0][0], ab[0][1], ab[1][0], ab[1][1]
    return (x1+x2)/2, (y1+y2)/2

In [115]:
def distance(p0,p1):
    return math.sqrt((p0[0] - p1[0])**2 + (p0[1] - p1[1])**2)

In [116]:
def assignToAlley(point,alleyCenters):
    """point is [x,y]
    alleyCenters is dict 1:[com_x, com_y] ... 17:[]
    """
    return sorted([(i,distance(point, alleyCenters[i])) for i in range(1,18)], key = lambda x: x[1])[0][0]

In [124]:
def calcPotentiations(visitfrs):
    """visit frs is a list of frs for each visit that we are looking at"""
    if visitfrs[0] == 0:
        return []
    else:
        return [i/visitfrs[0] for i in visitfrs]

In [260]:
alleyCenters = {i:[] for i in range(1,18)}
for i in range(1,18):
    center = calcAlleyCenter(Def.alleyBounds[i-1])
    alleyCenters[i] = center

In [329]:
reward_potentiations = [] # will be a list of lists with implicit ordering that they are in order of rewards for that alley
numvisits = 5 # number of visits to consider not including visit that got an r

#rewards_ = np.random.choice(rewards,len(rewards),replace=False)
rewards_ = rewards
avg_field_pots = []
for i,pf in enumerate(rm.PF):
    reward_potentiations = []
    contour = path.Path(list(zip(pf.perimeter[1]*binWidth+binWidth/2, pf.perimeter[0]*binWidth+binWidth/2)))
    PinC = unit.position[contour.contains_points(unit.position[:,1:])]
    posVisits = getVisits(PinC[:,0])
    field_FR = []
    field_TS = [] # take middle ts 
    for visit in posVisits:
        spk = unit.spikes[np.logical_and(unit.spikes[:,0] > visit[0], unit.spikes[:,0] < visit[-1])]
        field_FR.append((spk.shape[0]/len(visit))*30)
        field_TS.append(visit[0])
    
    com = [pf.com[1]*binWidth+binWidth/2, pf.com[0]*binWidth+binWidth/2]
    pf_alley = assignToAlley(com, alleyCenters)
    for r,rts in zip(rpos, rewards_):
        ralley = assignToAlley(r, alleyCenters)
        if ralley == pf_alley:
            nearestts = util.takeClosest([i[0] for i in posVisits], rts)
            visit_0 = [i[0] for i in posVisits].index(nearestts)
            pots = calcPotentiations(field_FR[visit_0:visit_0+numvisits])
            if len(pots) == numvisits:
                reward_potentiations.append(pots)
    avg = np.mean(np.asarray(reward_potentiations), axis=0)
    avg_field_pots.append(avg)

In [330]:
numvisits = 5 # number of visits to consider not including visit that got an r

#rewards_ = rewards
shuffmeans = np.empty((0,numvisits))
for q in range(1000):
    rewards_ = np.random.choice(rewards,len(rewards),replace=False)
    for i,pf in enumerate(rm.PF):
        reward_potentiations = [] 
        contour = path.Path(list(zip(pf.perimeter[1]*binWidth+binWidth/2, pf.perimeter[0]*binWidth+binWidth/2)))
        PinC = unit.position[contour.contains_points(unit.position[:,1:])]
        posVisits = getVisits(PinC[:,0])
        field_FR = []
        field_TS = [] # take middle ts 
        for visit in posVisits:
            spk = unit.spikes[np.logical_and(unit.spikes[:,0] > visit[0], unit.spikes[:,0] < visit[-1])]
            field_FR.append((spk.shape[0]/len(visit))*30)
            field_TS.append(visit[0])

        com = [pf.com[1]*binWidth+binWidth/2, pf.com[0]*binWidth+binWidth/2]
        pf_alley = assignToAlley(com, alleyCenters)
        for r,rts in zip(rpos, rewards_):
            ralley = assignToAlley(r, alleyCenters)
            if ralley == pf_alley:
                nearestts = util.takeClosest([i[0] for i in posVisits], rts)
                visit_0 = [i[0] for i in posVisits].index(nearestts)
                pots = calcPotentiations(field_FR[visit_0:visit_0+numvisits])
                if len(pots) == numvisits:
                    reward_potentiations.append(pots)

        rpots = np.array(reward_potentiations)
        rmean = np.nanmean(rpots, axis=0)
        try:
            shuffmeans = np.vstack((shuffmeans, rmean))
        except:
            pass

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: Mean of empty slice


### Characterizing Dynamic Fields in Repeating Cells (not nec that all dynamic fields obey repetition)

In [ ]:
field_duration = 3 # visits
fr_thresh = [0.2*max(field), 0.7*max(field)]

In [133]:
fields = []
times = []
for i,pf in enumerate(rm.PF):
    contour = path.Path(list(zip(pf.perimeter[1]*binWidth+binWidth/2, pf.perimeter[0]*binWidth+binWidth/2)))
    PinC = unit.position[contour.contains_points(unit.position[:,1:])]
    posVisits = getVisits(PinC[:,0])
    field_FR = []
    field_TS = [] # take middle ts 
    for visit in posVisits:
        spk = unit.spikes[np.logical_and(unit.spikes[:,0] > visit[0], unit.spikes[:,0] < visit[-1])]
        field_FR.append((spk.shape[0]/len(visit))*30)
        field_TS.append(visit[0])
    field_FR = util.weird_smooth(np.asarray(field_FR), 2)
    fields.append(field_FR)
    times.append(field_TS)
    
dynamic_list = [] #0 for no, 1 for yes
for j in range(len(fields)):
    dynamic_field = 0
    field, ts = fields[j], times[j]
    grad = np.gradient(field)
    zero_crossings = np.where(np.sign(grad[:-1]) != np.sign(grad[1:]))[0] + 1 # check if adjacent signs differ
    zero_crossings = np.hstack((0,zero_crossings, len(grad))) # add endpoints
    for i in range(len(zero_crossings)-1):
        roi = field[zero_crossings[i]:zero_crossings[i+1]]
        if max(roi) >= fr_thresh[1] and max(roi) <= fr_thresh[0] and (zero_crossings[i+1]-zero_crossings[i]>=field_duration):
            dynamic_field = 1
    dynamic_list.append(dynamic_field)

In [84]:
dynamic_list = [] #0 for no, 1 for yes
j=0
field, ts = fields[j], times[j]
grad = np.gradient(field)
zero_crossings = np.where(np.sign(grad[:-1]) != np.sign(grad[1:]))[0] + 1 # check if adjacent signs differ
zero_crossings = np.hstack((0,zero_crossings, len(grad))) # add endpoints
for i in range(len(zero_crossings)-1):
    roi = field[zero_crossings[i]:zero_crossings[i+1]]
    if max(roi) >= fr_thresh[1] and max(roi) <= fr_thresh[0] and (zero_crossings[i+1]-zero_crossings[i]>=field_duration):
        dynamic_list.append(1)
    else:
        dynamic_list.append(0)

11 19
19 24
24 26
26 28
28 34
34 40
40 43
43 46
46 52
